In [11]:
"""
爬数据的脚本我改了几个版本，这个是最新且最精简的，前几个版本你有兴趣可以看看，但用这个就可以
有几个知识点你了解一下基本就可以上手了
1. pandas的dataframe，包括表格信息处理（加减乘除，均值，数据类型，数据筛选等）和与其他文件类型互动（保存读取excel或csv）
2. numpy的array（dataframe的列叫做pd.series，本质上就是array）
3. 时间和日期的处理
4. beautifulsoup和request，用来交换网页信息
5. 一些基本的数据类型: list, dictionary, tuple等
6. 一些基本的工具：自定义公式，循环（for loop），试错（try except）
7. 遇到问题时有几个网站可以参考（比如你google一个问题，可以主要看这几个网站的答案），一般看几个例子就懂了：
    programiz
    geeksforgeeks
    csdn
    pandas，numpy等库的官网
"""

'\n爬数据的脚本我改了几个版本，这个是最新且最精简的，前几个版本你有兴趣可以看看，但用这个就可以\n有几个知识点你了解一下基本就可以上手了\n1. pandas的dataframe，包括表格信息处理（加减乘除，均值，数据类型，数据筛选等）和与其他文件类型互动（保存读取excel或csv）\n2. numpy的array（dataframe的列叫做pd.series，本质上就是array）\n3. 时间和日期的处理\n4. beautifulsoup和request，用来交换网页信息\n5. 一些基本的数据类型: list, dictionary, tuple等\n6. 一些基本的工具：自定义公式，循环（for loop），试错（try except）\n7. 遇到问题时有几个网站可以参考（比如你google一个问题，可以主要看这几个网站的答案），一般看几个例子就懂了：\n    programiz\n    geeksforgeeks\n    csdn\n    pandas，numpy等库的官网\n'

In [ ]:
"""

推荐每次操作前创建一个备份文件
"""

In [12]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
import datetime
import openpyxl
from colorama import Fore, Back, Style #print colored text
from time import sleep

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56',
    'Cookie': '_last_loginuname=viviyanwei; Hm_lvt_e14a1f00255ca6d026803cca20b08490=1607743514; href=https%3A%2F%2Finfo.mybxg.com%2Fm%2F20%2F1210%2F16%2F91D67724C337E319.html; accessId=5d36a9e0-919c-11e9-903c-ab24dbab411b; qimo_xstKeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=; _login_token=3191c6adf1eb4ce191bcbc278dc29301; _login_mid=2468603; _login_uid=1626926; _login_ip=116.228.72.50; 3191c6adf1eb4ce191bcbc278dc29301=11%3D5%261%3D10; Hm_lpvt_e14a1f00255ca6d026803cca20b08490=1607926822; qimo_seosource_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E5%85%B6%E4%BB%96%E7%BD%91%E7%AB%99; qimo_seokeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E6%9C%AA%E7%9F%A5; pageViewNum=9',
}
# User-Agent获取：打开edge浏览器，在地址栏输入edge://version/ 里面就有user agent的信息
# Cookie获取：用户名密码可以找李忆君(Yvonne)要，打开我的钢铁网首页，先登录，然后获取cookie可以参考这个帖子https://blog.csdn.net/MuWinter/article/details/75313476

In [13]:
#用来计算运行时间
program_start_time = datetime.datetime.now()

In [14]:
start_date = datetime.date(2021,4,9) #确定日期范围
end_date = datetime.date(2021,4,10) #日期范围为start_date到end_date-1，即不包含end_date当天的日期
page_range = 1 #选择页数
local_path = r'2019-now.xlsx' #如果保存路径与ipynb文件的地址一致，则只需要明确文件名，不需要路径

"""
将原材料分为2组，
dict_urls_1 在网站中的表格有 table header，即'th'
dict_urls_2 无 table header
"""
dict_urls = {
    'dict_urls_1': { 
    '无锡300系不锈钢冷轧卷板': 'https://list.info.mybxg.com/market/p-665,666,667,2222,2257,2259------0-01010301-------1.html', #原来叫‘无锡300系冷轧卷板’
    '无锡300系不锈钢热轧卷板': 'https://list.info.mybxg.com/market/p-671-0----0102160102-0-01010301-------1.html', #原来叫‘无锡300系热轧卷板’
    '300系不锈钢工业无缝管': 'https://list.info.mybxg.com/market/p-681-----0102160203-0--------1.html',
    '300系不锈钢工业焊管': 'https://list.info.mybxg.com/market/p-678-----010216020201-0--------1.html',
    '不锈钢角钢': 'https://list.info.mybxg.com/market/p-691-----0102160303-0--------1.html'
    },
    'dict_urls_2': { 
    '上海市场工角槽钢': 'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html',
    '上海市场冷轧板卷': 'https://list1.mysteel.com/market/p-221-----01010401-0-010101-------1.html',
    '上海市场热轧板卷': 'https://list1.mysteel.com/market/p-221-----01010301-0-010101-------1.html',
    '上海市场低合金卷': 'https://list1.mysteel.com/market/p-221-----01010302-0-010101-------1.html',
    '上海市场镀锌板卷': 'https://list1.mysteel.com/market/p-221-----01010501-0-010101-------1.html',
    '上海市场无缝管': 'https://list1.mysteel.com/market/p-221-----01010901-0-010101-------1.html',
    '沈阳市场H型钢': 'https://list1.mysteel.com/market/p-223-----01010705-0-01050101-------1.html',
    '上海市场方矩管': 'https://list1.mysteel.com/market/p-221-----01010904-0-010101-------1.html',
    '长春市场工角槽钢': 'https://list1.mysteel.com/market/p-223-----01010701-0-01050201-------1.html',
    '长春市场冷轧板卷': 'https://list1.mysteel.com/market/p-223-----01010401-0-01050201-------1.html',
    '烟台市场热轧开平板': 'https://list1.mysteel.com/market/p-223-----01010306-0-01010205-------1.html'
    }
}

dict_urls_1_list = list(dict_urls['dict_urls_1'].items())
dict_urls_2_list = list(dict_urls['dict_urls_2'].items())

In [15]:
#没用上，可以参考
def datetime_str_to_date (date_time_str):
    """
    convert dateimte str to dateimte to date
    input '%Y-%m-%d %H:%M'
    output %Y-%m-%d
    """
    try:
        date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
        date_ojb = date_time_obj.date()
    except TypeError:
        print('Type error!')
    else:
        return date_ojb

In [16]:
def get_n_pages_url (page1_url, n):
    """
    input first page url ending with '1.html'
    output list of url of n pages
    """
    page_n_list = []
    for i in range(1, n + 1):
        page_n_url = page1_url.replace('1.html', str(i) + '.html')
        page_n_list.append(page_n_url)
    return page_n_list

In [17]:
def get_title_info (input_page_list):
    """
    input a list of page url
    output a list of tile info 即（日期，title，链接）
    """
    page_list = []
    for url in input_page_list:
        response = requests.get(url)
        content = response.content
        soup = BeautifulSoup(content, "lxml")

        all_titles = soup.findAll('li', {"data-val":"0"})

        for i in range(len(all_titles)):
            txt = all_titles[i]
            url = 'https:' + txt.a.get('href')
            title = txt.a.get('title')
            date = datetime.datetime.strptime(txt.span.get_text(), '%Y-%m-%d %H:%M')
            page_list.append([title, date, url])

    return page_list

In [18]:
#后续读取excel文件中已有的sheet，防止overwrite，放在这里的运行时间比放在for loop里短
book = openpyxl.load_workbook(local_path)

In [19]:
"""dict1"""
for d1_item in dict_urls_1_list:
#     sleep(3) #在每次访问之间暂停几秒，防止网站访问的request被拒绝
    pages = get_n_pages_url(d1_item[1], n = page_range) #创建list，包含所有页数的链接
    titles = get_title_info(pages)
    title_list_filtered = [] #append 所有页数中标题的链接
    for i in range(len(titles)):
        if d1_item[0] in titles[i][0]: #筛选包含原材料名称的标题链接
            title_list_filtered.append(titles[i])
    print(Fore.YELLOW, d1_item) #print title
    df_list = [] #创建空list，用于后续将一个原材料的所有数据存入，并保存为dataframe
    try:
        for item in title_list_filtered: 
            title = item[0] #提取title_list_filtered中每一个item的title，date和链接
            date = item[1]
            url = item[2]

            response = requests.get(url, headers = headers)
            content = response.content
            soup = BeautifulSoup(content,'lxml') #解析网页数据

            table_header = soup.findAll('th')
            header_list = []
            for i in table_header:
                text = i.get_text()
                text_strip = text.strip('\r\n\t请筛选').strip()
                header_list.append(text_strip)

            table_body = soup.find('tbody')
            table_row = table_body.findAll('tr')
            for tr in table_row:
                td = tr.findAll('td')
                row_list = []
                for i in td:
                    text = i.get_text()
                    text_strip = text.strip('\r\n\t请筛选').strip()
                    row_list.append(text_strip)
                df_list.append([date,title, url] + row_list[:7]) #保留网页中表格的前7列，加上之前提取的title，date，链接3列

        df = pd.DataFrame(data = df_list, columns = ['日期','标题','网址'] + header_list[:7]) #保留header list的3+7列，与网页数据一致
        df = df[(df['日期'] >= str(start_date)) & (df['日期'] < str(end_date))] #日期筛选
        df = df.astype({str(np.array(df.columns)[df.columns.str.contains(pat = '价格')][0]): int}) #价格列数据类型转换为数值
        print(Fore.BLACK, df)
        df_old = pd.read_excel(local_path, sheet_name = d1_item[0], header = None) #读取excel对应sheet中已有数据
        writer = pd.ExcelWriter(local_path, mode = 'w', engine = 'openpyxl')
        writer.book = book
        writer.sheets = {ws.title: ws for ws in book.worksheets} #写入已有sheet，而不会覆盖
        df.to_excel(writer, index = False, header = None, sheet_name = d1_item[0], encoding = 'utf-8', startrow = len(df_old)) #从已有数据的下一行开始写入新数据
        writer.save()
        writer.close()
    except:
        print(Fore.RED, d1_item)

 ('无锡300系不锈钢冷轧卷板', 'https://list.info.mybxg.com/market/p-665,666,667,2222,2257,2259------0-01010301-------1.html')
                      日期                   标题  \
0   2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
1   2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
2   2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
3   2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
4   2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
..                  ...                  ...   
262 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
263 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
264 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
265 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
266 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   

                                                    网址  钢种             规格  \
0    https://info.mybxg.com/m/21/0409/09/262AB152DE...  卷板     0.4*1219*C   
1    https://info.mybxg.com/m/21/0409/09/262AB152DE...  卷板     0.5*1219*C   
2    https://info.mybxg.com/m/21/0409/09/262AB152DE...  卷板     0.6*1219*C   

In [20]:
"""dict2"""
book = openpyxl.load_workbook(local_path)

for d2_item in dict_urls_2_list:
#     sleep(3) #pause between requests
    pages = get_n_pages_url(d2_item[1], n = page_range)
    titles = get_title_info(pages)
    title_list_filtered = []
    for i in range(len(titles)):
        if d2_item[0] in titles[i][0]:
            title_list_filtered.append(titles[i])
    print(Fore.YELLOW, d2_item) #print title
    try:
        df_list = []
        for item in title_list_filtered:
            title = item[0]
            date = item[1]
            url = item[2]

            response = requests.get(url, headers = headers)
            content = response.content
            soup = BeautifulSoup(content,'lxml')

            table_body = soup.find('table',{'id':'marketTable'})
            table_row = table_body.findAll('tr')
            table_list=[]
            for tr in table_row:
                td = tr.findAll('td')
                row_list = []
                for i in td:
                    text = i.get_text()
                    text_strip = text.strip('\r\n\t请筛选').strip()
                    row_list.append(text_strip)
        #         df_list.append([date,title, url] + row_list[:7])
                table_list.append([date,title, url] + row_list[:7])
            header_list = table_list.pop(0) #剪切第一行，作为header
            table_list.pop(0)
            df_list.extend(table_list)
        df = pd.DataFrame(data = df_list, columns = ['日期','标题','网址'] + header_list[3:10]) #保留3+7列
        df = df[(df['日期'] >= str(start_date)) & (df['日期'] < str(end_date))]
        df = df.astype({str(np.array(df.columns)[df.columns.str.contains(pat = '价格')][0]): int}) #价格列数据类型转换为数值
        print(Fore.BLACK, df)
        df_old = pd.read_excel(local_path, sheet_name = d2_item[0], header = None)
        writer = pd.ExcelWriter(local_path, mode = 'w', engine = 'openpyxl')
        writer.book = book
        writer.sheets = {ws.title: ws for ws in book.worksheets}
        df.to_excel(writer, index = False, header = None, sheet_name = d2_item[0], encoding = 'utf-8', startrow = len(df_old))
        writer.save()
        writer.close()
    except:
        print(Fore.RED, d2_item)

 ('上海市场工角槽钢', 'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html')
                     日期              标题  \
0  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
1  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
2  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
3  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
4  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
..                 ...             ...   
65 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
66 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
67 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
68 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
69 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   

                                                   网址    品名   规格（mm）     材质  \
0   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  30*30*3  Q235B   
1   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  40*40*4  Q235B   
2   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  50*50*5  Q235B   
3   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  50*50*5  Q235B   
4   htt

In [22]:
header_list

[datetime.datetime(2021, 2, 2, 10, 21),
 '2日烟台市场热轧开平板价格行情',
 'https://rezha.mysteel.com/m/21/0202/10/19DACF3AEC0B1CDF.html',
 '品名',
 '规格（mm）',
 '材质',
 '钢厂/产地',
 '价格',
 '涨跌',
 '备注']

In [21]:
program_end_time = datetime.datetime.now()
print('总计运行时间: {}'.format(program_end_time - program_start_time))

总计运行时间: 0:48:12.518370
